1

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
smarket = sm.datasets.get_rdataset("Smarket", "ISLR").data

In [7]:
smarket.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,2001,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
1,2001,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2,2001,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
3,2001,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
4,2001,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


In [6]:
smarket.corr(numeric_only = 'True')

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today
Year,1.000000,0.029700,0.030596,0.033195,0.035689,0.029788,0.539006,0.030095
Lag1,0.029700,1.000000,-0.026294,-0.010803,-0.002986,-0.005675,0.040910,-0.026155
Lag2,0.030596,-0.026294,1.000000,-0.025897,-0.010854,-0.003558,-0.043383,-0.010250
Lag3,0.033195,-0.010803,-0.025897,1.000000,-0.024051,-0.018808,-0.041824,-0.002448
Lag4,0.035689,-0.002986,-0.010854,-0.024051,1.000000,-0.027084,-0.048414,-0.006900
Lag5,0.029788,-0.005675,-0.003558,-0.018808,-0.027084,1.000000,-0.022002,-0.034860
Volume,0.539006,0.040910,-0.043383,-0.041824,-0.048414,-0.022002,1.000000,0.014592
Today,0.030095,-0.026155,-0.010250,-0.002448,-0.006900,-0.034860,0.014592,1.000000


# Logistic Regression

In [13]:
# y
endog = (smarket['Direction'] == 'Up').astype('int64')

# X
# MAKE SURE YOU DO THIS IN THE HW
exog = sm.add_constant(smarket.drop(columns=['Direction', 'Year','Today']))

# Trues get coerced into 1s
# Falses now are 0s

In [14]:
logit_mod = sm.Logit(endog,exog).fit() # you need some sort of optimizer.
logit_mod.summary()

Optimization terminated successfully.
         Current function value: 0.691034
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              Direction   No. Observations:                 1250
Model:                          Logit   Df Residuals:                     1243
Method:                           MLE   Df Model:                            6
Date:                Tue, 06 Feb 2024   Pseudo R-squ.:                0.002074
Time:                        08:55:27   Log-Likelihood:                -863.79
converged:                       True   LL-Null:                       -865.59
Covariance Type:            nonrobust   LLR p-value:                    0.7319
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1260      0.241     -0.523      0.601      -0.598       0.346
Lag1          -0.0731      0.050     -1.457      0.145      -0.171       0.025
Lag2          -0.0423      0.050     -0.845      0.398      -0.140       0.056
Lag3           0.0111      0.050      0.222      0.824      -0.087       0.109
Lag4           0.0094      0.050      0.187      0.851      -0.089       0.107
Lag5           0.0103      0.050      0.208      0.835      -0.087       0.107
Volume         0.1354      0.158      0.855      0.392      -0.175       0.446
==============================================================================
"""

What do you do in the absense of a useful model
 - well it is better than a coin flip!

exp(b1) == 92%

- for every unit increase in lag 1, pr(up) decreases by 8% 

# Estimate Sample Error

### This is much more useful

#### We need to train/test split

In [17]:
train_mask = smarket['Year'] < 2005

In [18]:
testsmarket[~train_mask]

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,2001,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
1,2001,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2,2001,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
3,2001,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
4,2001,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up
...,...,...,...,...,...,...,...,...,...
993,2004,0.046,0.342,0.904,0.038,-0.749,0.9561,-0.431,Down
994,2004,-0.431,0.046,0.342,0.904,0.038,0.9220,0.715,Up
995,2004,0.715,-0.431,0.046,0.342,0.904,0.9830,-0.007,Down
996,2004,-0.007,0.715,-0.431,0.046,0.342,0.9259,0.008,Up


In [32]:
train_endog = (smarket.loc[train_mask,'Direction'] == 'Up').astype('int64')
train_exog = exog = sm.add_constant(smarket[train_mask].drop(columns=['Direction', 'Year','Today']))

test_endog = (smarket.loc[~train_mask,'Direction'])
test_exog = exog = sm.add_constant(smarket[~train_mask].drop(columns=['Direction', 'Year','Today']))


In [33]:
logit_mod = sm.Logit(train_endog, train_exog)
logit_res = logit_mod.fit()

logit_res.summary()

Optimization terminated successfully.
         Current function value: 0.691936
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:              Direction   No. Observations:                  998
Model:                          Logit   Df Residuals:                      991
Method:                           MLE   Df Model:                            6
Date:                Tue, 06 Feb 2024   Pseudo R-squ.:                0.001562
Time:                        09:21:59   Log-Likelihood:                -690.55
converged:                       True   LL-Null:                       -691.63
Covariance Type:            nonrobust   LLR p-value:                    0.9044
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1912      0.334      0.573      0.567      -0.463       0.845
Lag1          -0.0542      0.052     -1.046      0.295      -0.156       0.047
Lag2          -0.0458      0.052     -0.884      0.377      -0.147       0.056
Lag3           0.0072      0.052      0.139      0.889      -0.094       0.108
Lag4           0.0064      0.052      0.125      0.901      -0.095       0.108
Lag5          -0.0042      0.051     -0.083      0.934      -0.104       0.096
Volume        -0.1163      0.240     -0.485      0.628      -0.586       0.353
==============================================================================
"""

In [37]:
logit_test_probs = logit_res.predict(test_exog)
logit_test_preds = pd.Series(logit_test_probs).apply(lambda x: 'Up' if (x >.5) else 'Down')

In [41]:
pd.DataFrame(
    confusion_matrix(test_endog,logit_test_preds),
    columns=['PredDown','PredUp'],
    index=['ActDown','ActUp']
)

,PredDown,PredUp
ActDown,77,34
ActUp,97,44


In [42]:
(logit_test_preds == test_endog).mean()

0.4801587301587302

In [43]:
# or error 
(logit_test_preds != test_endog).mean()

0.5198412698412699

# LDA

In [47]:
lda_clf = LinearDiscriminantAnalysis(
    store_covariance=True,
)

lda_clf.fit(train_exog,train_endog)

LinearDiscriminantAnalysis(store_covariance=True)

In [50]:
pd.Series(
    lda_clf.priors,
    index = lda_clf.classes_
)

C:\Users\dansc\AppData\Local\Temp\ipykernel_4616\346572650.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  pd.Series(


0   NaN
1   NaN
dtype: float64

In [54]:
pd.DataFrame(
    lda_clf.means_,
    index = lda_clf.classes_,
    columns = train_exog
)

ValueError: Index data must be 1-dimensional

In [55]:
pd.Series(lda_clf.coef_.flatten(),index=lda_clf.classes_)

ValueError: Length of values (7) does not match length of index (2)

In [70]:
lda_pred = ['Up' if x == 1 else 'Down' for x in lda_clf.predict(test_exog)]

In [71]:
# dif from predict
lda_clf.score(
    test_exog,
    lda_pred
)

C:\Users\dansc\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:219: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


0.0

In [74]:
pd.DataFrame(
    confusion_matrix(test_endog,lda_pred),
)

,0,1
0,77,34
1,97,44


## QDA 

In [76]:
qda_clf = QuadraticDiscriminantAnalysis()
qda_clf.fit(train_exog, train_endog)

C:\Users\dansc\AppData\Roaming\Python\Python39\site-packages\sklearn\discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis()

In [77]:
qda_pred = ['Up' if x == 1 else 'Down' for x in qda_clf.predict(test_exog)]

C:\Users\dansc\AppData\Roaming\Python\Python39\site-packages\sklearn\discriminant_analysis.py:960: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\dansc\AppData\Roaming\Python\Python39\site-packages\sklearn\discriminant_analysis.py:960: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\dansc\AppData\Roaming\Python\Python39\site-packages\sklearn\discriminant_analysis.py:963: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])


In [82]:
qda_clf.score(
    test_exog,
    qda_pred
)

C:\Users\dansc\AppData\Roaming\Python\Python39\site-packages\sklearn\discriminant_analysis.py:960: RuntimeWarning: divide by zero encountered in power
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\dansc\AppData\Roaming\Python\Python39\site-packages\sklearn\discriminant_analysis.py:960: RuntimeWarning: invalid value encountered in multiply
  X2 = np.dot(Xm, R * (S ** (-0.5)))
C:\Users\dansc\AppData\Roaming\Python\Python39\site-packages\sklearn\discriminant_analysis.py:963: RuntimeWarning: divide by zero encountered in log
  u = np.asarray([np.sum(np.log(s)) for s in self.scalings_])
C:\Users\dansc\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:219: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


0.0

## KNN

In [80]:
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(train_exog, train_endog)

KNeighborsClassifier(n_neighbors=1)

In [83]:
knn1_pred = ['Up' if x == 1 else 'Down' for x in knn1.predict(test_exog)]
knn1.score(
    test_exog,
    lda_pred
)

C:\Users\dansc\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:219: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred


0.0

## Caravan Dataset

In [86]:
caravan_filepath = 'https://raw.githubusercontent.com/JWarmenhoven/ISLR-python/master/Notebooks/Data/Caravan.csv'
caravan = pd.read_csv(caravan_filepath, index_col = "Unnamed: 0")
caravan.head()

,MOSTYPE,MAANTHUI,MGEMOMV,MGEMLEEF,MOSHOOFD,MGODRK,MGODPR,MGODOV,MGODGE,MRELGE,...,APERSONG,AGEZONG,AWAOREG,ABRAND,AZEILPL,APLEZIER,AFIETS,AINBOED,ABYSTAND,Purchase
1,33,1,3,2,8,0,5,1,3,7,...,0,0,0,1,0,0,0,0,0,No
2,37,1,2,2,8,1,4,1,4,6,...,0,0,0,1,0,0,0,0,0,No
3,37,1,2,2,8,0,4,2,4,3,...,0,0,0,1,0,0,0,0,0,No
4,9,1,3,3,3,2,3,2,4,5,...,0,0,0,1,0,0,0,0,0,No
5,40,1,4,2,10,1,4,1,4,7,...,0,0,0,1,0,0,0,0,0,No


In [88]:
caravan['Purchase'].value_counts()
# super duper unbalanced!

No     5474
Yes     348
Name: Purchase, dtype: int64

In [93]:
(caravan['Purchase']=='Yes').mean() # 6% are yesses

0.05977327378907592

In [95]:
caravan.T.head() #some vars are much larger than others.

,1,2,3,4,5,6,7,8,9,10,...,5813,5814,5815,5816,5817,5818,5819,5820,5821,5822
MOSTYPE,33,37,37,9,40,23,39,33,33,11,...,40,3,1,13,3,36,35,33,34,33
MAANTHUI,1,1,1,1,1,1,2,1,1,2,...,1,1,1,1,1,1,1,1,1,1
MGEMOMV,3,2,2,3,4,2,3,2,2,3,...,3,2,3,2,2,1,4,3,3,3
MGEMLEEF,2,2,2,3,2,1,2,3,4,3,...,4,4,3,3,3,2,4,4,2,3
MOSHOOFD,8,8,8,3,10,5,9,8,8,3,...,10,1,1,3,1,8,8,8,8,8


In [108]:
#bc of this let us standardize
# standarizing means modeling on the z-score
standardize_X = caravan.drop(columns = ['Purchase']).transform(lambda x: (x- x.mean() / x.std()))

In [110]:
standardize_X['MOSTYPE'].mean() # this didn't work... @$@

22.365445186788175

In [112]:
test_mask = range(1,1001)
X_train = standardize_X.drop(index = test_mask)
X_test = standardize_X.loc[test_mask,]
y_train = caravan.drop(index = test_mask)['Purchase']
y_test = caravan.loc[test_mask, 'Purchase']

In [113]:
knn1_clf = KNeighborsClassifier(n_neighbors=1)
knn1_clf.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=1)

#### Make Preds